Feature extraction package import

In [ ]:
import sensormotion as sm

Numpy and pandas are libraries for working with advanced multidimensional arrays

In [2]:
import numpy as _np


In [7]:
import pandas as _pd

Defining simple feature functions

In [10]:
def imu_feature_simple(filename):
    ave, full = imu_features(filename);
    return ave 

Defining full feature functions

In [15]:
def imu_feature_full(filename):
    ave, full = imu_features(filename);
    return full

The major function it returns both simple and full feature functions. The imported sensormotion package is not perfect, so it needs a lot of walk arounds to detect the warnings. The commands are used to create an excel spreadsheet. 

A filter is used to eliminate the noise and to make the output signals smoother

Going through sensor 1, 2 and 3 and to extract the accelerometer values.



In [22]:
def imu_features(filename):
    import warnings
    warnings.filterwarnings('error')
    feature_number = 16
    feature_index = 0
    try:
        fileIO = _pd.ExcelFile(filename)
        print(filename)
            
    except:
        if not fileIO:
            fileIO.close()
            
    b1, a1 = sm.signal.build_filter(frequency=5,
                                  sample_rate=100,
                                  filter_type='low',
                                  filter_order=4)

Read from the excel sheets the data from the nine rows and extract the time and corresponding acceleration values.

After getting these values, extract the vector magnitude of the acceleration 

From this find, the mean of the vector magnitude of the accleration and its corresponding standard deviation.

In [ ]:
for sheet_index in range(0, 3):
        try:
            
            df = _pd.read_excel(fileIO, sheet_name = sheet_index, header=None)
            
            
            t = _np.array(df.values[:,0], dtype=_np.float64)
            x = _np.array(df.values[:,7], dtype=_np.float64)
            y = _np.array(df.values[:,8], dtype=_np.float64)
            z = _np.array(df.values[:,9], dtype=_np.float64)
            
            
            temp = _np.multiply(x,x) + _np.multiply(y,y) + _np.multiply(z,z)             
            vec_sum = _np.sqrt(temp)            
            acc = x
            acc = _np.vstack([acc, y])
            acc = _np.vstack([acc, z])
            acc = _np.vstack([acc, vec_sum])
            
            amplitute_mean = _np.zeros(4)
            amplitute_std = _np.zeros(4)
            
            for i in range (0,4):
                amplitute_mean[i] = _np.mean(acc[i])
                amplitute_std[i] = _np.mean(acc[i])
            
            acc_filterd = []
            
            for i in range (0,4):
                if i == 0:
                    #first line we fill in
                    acc_filterd = sm.signal.filter_signal(b1, a1, signal=acc[i,:])
                else:
                    #then we add row by row
                    acc_filterd = _np.vstack([acc_filterd, sm.signal.filter_signal(b1, a1, signal=acc[i,:])])
                    
            cadence =_np.zeros(4)
            
            step_mean = _np.zeros(4)
            
            for i in range (0,4):
                peak_times, peak_values = sm.peak.find_peaks(time=t, signal=acc_filterd[i],
                                                         peak_type='valley',
                                                         min_val=0.2, min_dist=1,
                                                         plot=False)
                step_mean[i], step_sd, step_cov = sm.gait.step_time(peak_times = peak_times)
            
                cadence[i] = sm.gait.cadence(time=t, peak_times = peak_times, time_units='s')
                
            feature_index = 0
            
            for i in range (0,4):
                features[sheet_index, feature_index]=amplitute_mean[i]
                feature_index =  feature_index + 1
            
            for i in range (0,4):
                features[sheet_index, feature_index]=cadence[i]
                feature_index =  feature_index + 1
            
            for i in range (0,4):
                features[sheet_index, feature_index]=step_mean[i]
                feature_index =  feature_index + 1
                
            for i in range (0,4):
                features[sheet_index, feature_index]=amplitute_std[i]
                feature_index =  feature_index + 1
                
        except Warning:
            pass
                

In [ ]:
ave_features = _np.array([features[0,3],\
                             features[0,7],\
                             features[0,11],\
                             features[1,3],\
                             features[1,7],\
                             features[1,11],\
                             features[2,3],\
                             features[2,7],\
                             features[2,11]])
fileIO.close()
    
    return ave_features, features;

In [33]:
import os

In [ ]:
working_dir = os.path.dirname(os.path.abspath(__file__))

Load the dataset

In [ ]:
dataset = []


Label the data based on 0,1 and 2 commands

In [ ]:
for r, d, f in os.walk(data_set_a):
    for file in f:
        if '.xls' in file:
            dataset.append({ "filename": data_set_a + file, "label": 0 })
            
for r, d, f in os.walk(data_set_b):
    for file in f:
        if '.xls' in file:
            dataset.append({ "filename": data_set_b + file, "label": 1 })

for r, d, f in os.walk(data_set_c):
    for file in f:
        if '.xls' in file:
            dataset.append({ "filename": data_set_c + file, "label": 2 })

Get all the features from the data

In [ ]:
all_data   = []
all_labels = []


feature extraction from all the data

In [ ]:
for item in dataset:
    #print(item['filename'])
    all_data.append(imu_feature_simple(item['filename']))
    all_labels.append(item['label'])

Change the data type from the list to array

In [ ]:
all_data = _np.asarray(all_data)
all_labels = _np.asarray(all_labels)

Here we fill all Nan as the mean of the column
Obtain mean of columns as you need, nanmean is just convenient

In [ ]:
col_mean = _np.nanmean(all_data, axis=0)

Find indices that you need to replace

In [ ]:
inds = _np.where(_np.isnan(all_data))

Place column means in the indices. Align the arrays using take

In [ ]:
all_data[inds] = _np.take(col_mean, inds[1])

Save the data

In [ ]:
_np.savez("data_1",all_data=all_data,all_labels=all_labels)